## Transcribing the YouTube Video
The context we want to send the model comes from a YouTube video. Let's download the video and transcribe it using [OpenAI's Whisper](https://openai.com/index/whisper/).

In [ ]:
from pathlib import Path
import tempfile
import whisper
from yt_dlp import YoutubeDL

YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=cdiD-9MMpb0"
TEMP_DIR = Path("..") / Path("temp")
TRANSCRIPT_FILE = (Path("..") / Path("transcription.txt"))

if not TRANSCRIPT_FILE.exists():
    # Create a temporary directory
    with tempfile.TemporaryDirectory(dir=TEMP_DIR) as temp_dir:
        # Define the URL of the YouTube video

        # Set up yt-dlp options
        ydl_opts = {
            "verbose": True,
            "format": "m4a/bestaudio/best",
            "outtmpl": str(Path(temp_dir) / Path("%(title)s.%(ext)s" )),
        }

        # Download the audio
        with YoutubeDL(ydl_opts) as ydl:
            error_code = ydl.download([YOUTUBE_VIDEO])
            if error_code != 0:
                raise Exception("Error downloading the video")

        # Load whisper model
        whisper_model = whisper.load_model("base")

        # Foreach audio file in the temporary directory
        for file in Path(temp_dir).iterdir():
            if file.suffix == ".m4a":
                # Transcribe the audio
                transcription = whisper_model.transcribe(str(file), fp16=False, verbose=True)["text"].strip()
            
            # Write the transcription to a file
            with TRANSCRIPT_FILE.open("w") as file:
                    file.write(transcription)